## Segmenting and Clustering Neighbourhoods in Toronto

### Part 1: Converting Wikipedia html table into a DataFrame

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml

In [2]:
source_file='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(source_file)

#### 1.1 Read the web html file with BeautifulSoup 

In [3]:
page_content=bsoup(r.text,"html.parser")
#page_content

#### 1.2 Find Table in html file

In [5]:
mytable=page_content.table
#mytable

In [6]:
results=mytable.find_all('tr')
no_lines=len(results)
print('now we have found', no_lines, ' lines incl. the Header, which is the first line with 0 index')

now we have found 288  lines incl. the Header, which is the first line with 0 index


In [7]:
header=results[0].text.split()
header

['Postcode', 'Borough', 'Neighbourhood']

#### 1.3 Create a loop to extract the table contents into a dataframe df1

In [8]:
lines =[]
n=1
while n < no_lines :
    Postcode=results[n].text.split('\n')[1]
    Borough=results[n].text.split('\n')[2]
    Neighborhood=results[n].text.split('\n')[3]
    lines.append((Postcode, Borough,Neighborhood))
    n=n+1

df1=pd.DataFrame(lines, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df1.head(5)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
df1.shape

(287, 3)

#### 1.4 Remove all rows with Borough = 'Not assigned' 

In [15]:
df1[df1['Borough']=='Not assigned'].count()

PostalCode       77
Borough          77
Neighbourhood    77
dtype: int64

In [9]:
df1.drop(df1[df1.Borough =='Not assigned'].index, inplace = True)
df1.reset_index(drop=True, inplace = True)

In [17]:
df1.head(5)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [18]:
df1.tail()

,PostalCode,Borough,Neighbourhood
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West
209,M8Z,Etobicoke,South of Bloor


In [19]:
df1.shape

(210, 3)

#### 1.5 Replace 'Not assigned' neighborhoods with the name of the Borough

In [20]:
df1[df1['Neighbourhood']=='Not assigned'].count()

PostalCode       1
Borough          1
Neighbourhood    1
dtype: int64

In [10]:
df1.loc[df1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df1['Borough']

In [11]:
df1[df1['Neighbourhood']=='Not assigned'].count()

PostalCode       0
Borough          0
Neighbourhood    0
dtype: int64

##### 1 row with 'not assigned' Neighborhood was replaced by Borough

In [23]:
df1.shape

(210, 3)

#### 1.6 Combine into one row all PostalCodes/ Boroughs with several neighborhoods with the neighborhoods separated with a comma

In [12]:
print('Target Data Frame should only consists of ', len(df1.groupby(['PostalCode','Borough'])), ' lines')


Target Data Frame should only consists of  103  lines


In [13]:
df1 = df1[~df1['Borough'].isnull()]  # to filter out bad rows

In [39]:
df1.shape

(210, 3)

In [14]:
df_target = df1.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()

In [45]:
df_target.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [46]:
df_target.shape

(103, 3)

### Now we have the first part of Target Data Frame ready with only 103 lines

### End of Part 1 of 2 of Toronto Project --------------------------

### Part 2: Get the latitude and the longitude coordinates of each neighborhood

#### 2.1 Get the data from Geodata URL

In [15]:
url='http://cocl.us/Geospatial_data'
geodata_df=pd.read_csv(url)
geodata_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
geodata_df.shape

(103, 3)

In [16]:
geodata_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geodata_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 2.2 Merge the target Data Frame with Geodata

In [21]:
df_merged = df_target.join(geodata_df.set_index('PostalCode'), on='PostalCode')
#df_merged

### End of Part 2 of 2 of Toronto Project ------------------------

### Part 3: Exploring Toronto Neighbourhood

#### 3.1 Select Data Frame with Toronto boroughs

In [23]:
Toronto_data = df_merged[df_merged['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
Toronto_data.head(5)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [24]:
Toronto_data.shape


(38, 5)

#### Now we have 38 lines (37 without header) in Toronto Data Frame

In [32]:
Toronto_data.tail()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191
34,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763
35,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325
36,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


#### 3.2 Create a map of Toronto

In [27]:
latitude = 43.651571
longitude= -79.484450

In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [33]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 3.3 Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
Toronto_data.loc[0, 'Neighborhood']

In [ ]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#### 3.4 Create a function to repeat the same process to all the neighborhoods in Toronto Dataframe

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

##### let's check the size of the resulting dataframe

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

##### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


In [ ]:
toronto_venues.groupby('Neighborhood').count()

#### 3.5 Analyze Each Neighborhood

In [ ]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]

# rename Neighborhood for Neighbourhood so that future merge works
toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)
toronto_onehot.head()

##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

##### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
um_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

##### Let's put that into a *pandas* dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

#### 3.6 Cluster Neighbourhoods

###### Run *k*-means to cluster the neighborhood into 5 clusters

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

#### 3.7 Final resulting clusters visualization - Toronto Neighborhoods

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3.8 Examine Clusters

##### Cluster 1:

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

##### Cluster 2:

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

##### Cluster 3:

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

##### Cluster 4:

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

##### Cluster 5:

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]